In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dfhelper as dfh
import text_preproc as tp

In [ ]:
from transformers import AutoTokenizer, TFAutoModel

In [ ]:
tf.config.list_physical_devices('GPU')


In [219]:
pre_df = pd.read_pickle('dataset/train_df_v2.pkl')

In [220]:
pre_df.shape

(5000, 5)

In [221]:
# # taking only first thousand entries
# pre_df = pre_df[:1000]

In [222]:
pre_df.shape

(5000, 5)

In [223]:
dfh.printColNames(pre_df)

0 TITLE | 1 BULLET_POINTS | 2 DESCRIPTION | 3 PRODUCT_TYPE_ID | 4 PRODUCT_LENGTH | 

In [224]:
marr = pre_df.columns.values

title = marr[0]
bull_pt = marr[1]
desc = marr[2]
prod_type_id= marr[3]
prod_len = marr[4]


In [225]:
cpy_df = pre_df.copy()

In [226]:
# pre_df = pre_df.reset_index()

In [227]:
cpy_df = pd.get_dummies(pre_df, columns=[prod_type_id])

In [228]:
cpy_df
cpy_df.shape

(5000, 1622)

In [229]:
first_thousand = pre_df[:1000]
first_thousand

,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
1896156,Bildos 3Ply Non-Woven Fabric Disposable Surgic...,"[Care Instructions: Non Washable,Mask Has 3 La...","Face masks may also be called isolation, laser...",12591,669.29
1715725,Negaor 10 PCS Interior Door Card Trim Panel Re...,[* Interior door card trim panel clips fastene...,Please check the Manufacturer Part Number and ...,7305,314.96
229576,"Madden Girl Women's Domain Boot, Black Fabric,...",Chukka-inspired boot with four-eye lacing syst...,Suede wedge laceup bootie,3293,500.00
670308,Patel's Velocity perfume (60ml)(pack of 1),"[Long Lasting Fragrances,Made From Real and Na...",Patel's Velocity Perfume is Long Lasting Perfu...,254,157.48
945067,WorldCare® 1000W Car Power ply Output Socket U...,"[New & Imported Item,Delivers within 3 to 5 we...",WorldCare 1000W Car Power ply Output Socket US...,2087,590.55
...,...,...,...,...,...
165557,"Meenu Arts Flash X Hazard Module, Blinker/Flas...","[Plug N Play, 60 Patterns, No Switch Requried,...",FLASHER,7989,787.40
1497881,Leveret Fleece Mens Sleep Pants Elephant Small,"[100% Micro Fleece Polyester,All styles are ma...",You'll love these comfy fleece sleep pants fro...,2842,600.00
1511844,StaySoft Polarized Replacement Lenses for Oakl...,"[Perfect Outdoor Goods - For golf, cycling, ru...","<b>Stay here, Keep Soft!</b><br><br> That's St...",2799,400.00
189620,GRAFIQE Back Cover for ViVO Y20A V2052 Itachi ...,[GRAFIQE BACK COVER Great print Quality gives ...,"Vivo Y20A Back Cover by GRAFIQE Stylish, Ultra...",12057,236.22


In [230]:
import keyboard

In [231]:
# for i in range(1000):
#     print(pre_df.iloc[i]["TITLE"])
#     keyboard.wait('/')

In [ ]:
mlist = []

In [ ]:
# mdf = pre_df.drop(columns=['BULLET_POINTS', 'DESCRIPTION', 'PRODUCT_LENGTH'])
# mdf.head()

In [ ]:
mlist = []
# back prop
# we are obviously smart
# let's see how we predict and the try to extract
# print all with prod_len between 0 and 100
def printCountInGivenRange(pre_df, inr, outr):
    count = 0
    # for i in range(pre_df.shape[0]):
    for i in range(1000):
        if pre_df.iloc[i][prod_len] <= outr and pre_df.iloc[i][prod_len] >= inr:
            count += 1
            print(i, pre_df.iloc[i][prod_len], pre_df.iloc[i][title])
            mlist.append(i)
            num = ""
            for j in pre_df.columns.values:
                if ("id" in j.lower()):
                    num += str(pre_df.iloc[i][j]) + " "
            print(num)
                    



In [ ]:
# printCountInGivenRange(first_thousand, 0, 100)

In [ ]:
# printCountInGivenRange(first_thousand, 100, 200)

In [ ]:
# pre_df.max()

In [ ]:
# # test_mrow = pre_df[prod_len]
# for i,v in first_thousand.iterrows():
#     if (v[prod_len] == 1885801400.0):
#         print(v[title])
#         print(v)

In [ ]:
# simply put through lazy predict bus

In [ ]:
# # use cpy_df. First do splits and then use lazy predict
# %pip install lazypredict

In [ ]:
# from lazypredict.Supervised import LazyClassifier
# from sklearn.datasets import load_breast_cancer
# from sklearn.model_selection import train_test_split

# data = load_breast_cancer()
# X = data.data
# y= data.target

# X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.5,random_state =123)

# clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
# models,predictions = clf.fit(X_train, X_test, y_train, y_test)

# print(models)

In [ ]:
# %pip install gensim

In [ ]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# import gensim
# from lazypredict.Supervised import LazyClassifier

In [ ]:
# import re
# import string
# from nltk.tokenize import word_tokenize
# from nltk.corpus import stopwords
# from nltk.stem.wordnet import WordNetLemmatizer
# import gensim

# # Download the required NLTK resources
# import nltk
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

# def preprocess_text(text):
#     # Convert text to lowercase
#     text = text.lower()

#     # Remove numbers and punctuation
#     text = re.sub(r'\d+', '', text)
#     text = text.translate(str.maketrans('', '', string.punctuation))

#     # Tokenize text into words
#     words = word_tokenize(text)

#     # Remove stop words
#     stop_words = set(stopwords.words('english'))
#     words = [word for word in words if word not in stop_words]

#     # Lemmatize words
#     lem = WordNetLemmatizer()
#     words = [lem.lemmatize(word, "v") for word in words]

#     # Join words back into a sentence
#     preprocessed_text = " ".join(words)

#     return preprocessed_text

# # Example usage:
# text = "This is an example sentence. It will be used to demonstrate text preprocessing for Word2Vec."
# preprocessed_text = preprocess_text(text)
# print(preprocessed_text)

In [232]:
# Load data
# data = pd.read_csv('your_data.csv')
data = cpy_df.copy()


In [ ]:

# # Load and prepare word embeddings
# # w2v_model = Word2Vec.load('your_word_embeddings.model')
# w2v_model = gensim.models.KeyedVectors.load_word2vec_format('models/GoogleNews-vectors-negative300.bin.gz', binary=True)


In [ ]:
# # maybe you need to pre process the data
# # maybe you need to use ann and tensorflow instead of sklearn
# m_word = X_train[title].values[0].split(" ")[0]
# print(m_word)

In [ ]:
# w2v_model
# word = 'king'
# vector = w2v_model[m_word]
# print(vector)

In [ ]:
# %pip install transformers

In [ ]:
# Load pre-trained model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModel.from_pretrained(model_name)

# (batch_size, sequence_length, hidden_size) token ka embedding


In [ ]:

# # Tokenize input text and convert to TensorFlow tensors
# # text = "This is an example sentence."
# text = "This is an example sentence. Tokenize input text and convert to TensorFlow tensors "
# # inputs = tokenizer(text, return_tensors='tf')
# inputs = tokenizer(text, return_tensors='tf', max_length=512, truncation=True)
# # inputs = tokenizer(text)

# # Pass input through the model to get the output vector
# # outputs = model(inputs)[0]
# outputs = model(inputs)

# # Print the output vector
# print(outputs[0])

In [ ]:
# # Example list of texts
# # texts = ['This is the first text.', 'This is the second text.', 'This is the third text.']
# texts = ["HACASO - The Princess Sleeps Here- Pink Wall Decal Girl's Bedroom Decor", 'PINKCITY CREATION Handmade Showpieces Figurine Fluorite Stone Religious Idol Ganesha 5 cm.', 'Lovely Furniture Floating Mounted Cube Shape Wall Shelves Set of 4 (Black)']

# # Tokenize the texts
# # tokenized_texts = tokenizer(texts, padding=True, truncation=True, return_tensors='tf', max_length=512)
# tokenized_texts = tokenizer(texts, padding=True, truncation=True, return_tensors='tf', max_length=512)
# outputs = model(inputs)
# # Print the tokenized texts
# print(outputs)

In [ ]:
# # text = X_train[title].values[0]
# text = X_train[title].values[0:3].tolist()
# print(type(text))
# print(text)


In [ ]:
# # inputs = tokenizer(text, return_tensors='tf')
# inputs = tokenizer(text, return_tensors='tf', padding=True, truncation=True, max_length=512)
# # inputs = tokenizer(text)

# # Pass input through the model to get the output vector
# outputs = model(inputs)

# # Print the output vector
# print(outputs[0])

In [ ]:
# # Convert output tensor to numpy array
# output_array = outputs.numpy()

# # Flatten and reshape the output array to match sklearn input format
# output_array = output_array.flatten().reshape(1, -1)

# # Print the output array
# print(output_array)
# output_array.shape

In [ ]:

# msent = preprocess_text(msent)
# print(msent)
# msent = msent.split(" ")
# print(msent)
# # msent = [w2v_model[w] for w in msent]
# msent2 = []
# for i in msent:
#     try:
#         msent2.append(w2v_model[i])
#     except Exception as e:
#         print(e)
#         print(i)

# print(msent2)

In [ ]:

# msent = preprocess_text(msent)
# print(msent)
# msent = msent.split(" ")
# print(msent)
# # msent = [w2v_model[w] for w in msent]
# msent2 = []
# for i in msent:
#     try:
#         msent2.append(w2v_model[i])
#     except Exception as e:
#         print(e)
#         print(i)

# print(msent2)

In [233]:
def mtokenize3(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors='tf', max_length=512)
    outputs = model(inputs)
    
    # print(outputs.pooler_output)
    # print(outputs.last_hidden_state)
    # contextual_embeddings, pooled_output = outputs

  
    return outputs.pooler_output

In [ ]:

# X_train_embeddings = [w2v_model.wv[word] for word in X_train[title]]
# X_test_embeddings = [w2v_model.wv[word] for word in X_test[title]]


In [234]:
# importing train_test_split
from sklearn.model_selection import train_test_split

In [235]:
# # data[title] = data[title] + " " + data[desc] + " " + data[bull_pt]
# # data[title] = data[desc]
# data[title] = data[bull_pt] 
# # try the above thing next time


In [236]:
data[title].values[0]
# data[bull_pt].values[0]

'Bildos 3Ply Non-Woven Fabric Disposable Surgical Dust Mask With Nose Clip Maximum Protection (Blue, Without Valve, Pack of 200) for Unisex'

In [237]:
data.head()

,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_LENGTH,PRODUCT_TYPE_ID_0,PRODUCT_TYPE_ID_1,PRODUCT_TYPE_ID_2,PRODUCT_TYPE_ID_4,PRODUCT_TYPE_ID_5,PRODUCT_TYPE_ID_6,...,PRODUCT_TYPE_ID_13248,PRODUCT_TYPE_ID_13279,PRODUCT_TYPE_ID_13280,PRODUCT_TYPE_ID_13290,PRODUCT_TYPE_ID_13294,PRODUCT_TYPE_ID_13343,PRODUCT_TYPE_ID_13346,PRODUCT_TYPE_ID_13347,PRODUCT_TYPE_ID_13368,PRODUCT_TYPE_ID_13414
1896156,Bildos 3Ply Non-Woven Fabric Disposable Surgic...,"[Care Instructions: Non Washable,Mask Has 3 La...","Face masks may also be called isolation, laser...",669.29,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1715725,Negaor 10 PCS Interior Door Card Trim Panel Re...,[* Interior door card trim panel clips fastene...,Please check the Manufacturer Part Number and ...,314.96,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
229576,"Madden Girl Women's Domain Boot, Black Fabric,...",Chukka-inspired boot with four-eye lacing syst...,Suede wedge laceup bootie,500.00,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
670308,Patel's Velocity perfume (60ml)(pack of 1),"[Long Lasting Fragrances,Made From Real and Na...",Patel's Velocity Perfume is Long Lasting Perfu...,157.48,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
945067,WorldCare® 1000W Car Power ply Output Socket U...,"[New & Imported Item,Delivers within 3 to 5 we...",WorldCare 1000W Car Power ply Output Socket US...,590.55,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [238]:
# X_train[title] = X_train[title] + " " + X_train[desc] + " " + X_train[bull_pt]
# print(X_train[title].values[0])

In [239]:
# # Example input text
# input_text = X_train[title].values.tolist()

# # Convert each sentence to tokens and input IDs
# input_ids = []
# for text in input_text:
#     # tokens = tokenizer.tokenize(text)
#     tokens = tokenizer.tokenize(text, padding=True, truncation=True, return_tensors='tf', max_length=512)
#     ids = tokenizer.convert_tokens_to_ids(tokens)
#     input_ids.append(ids)

# # Create a `tf.data.Dataset` object to feed the input data to the model
# input_ids = tf.keras.preprocessing.sequence.pad_sequences(input_ids, maxlen=128, padding='post', truncating='post')
# dataset = tf.data.Dataset.from_tensor_slices((input_ids,))
# dataset = dataset.batch(2)

In [240]:
# # Pass the input data through the pre-trained model to get the embeddings
# all_embeddings = []
# for batch in dataset:
#     with tf.device('/GPU:0'):
#         outputs = model(batch)
#         embeddings = outputs.pooler_output
#         all_embeddings.append(embeddings.numpy())

# # Concatenate the embeddings for all the batches
# all_embeddings = np.concatenate(all_embeddings, axis=0)

In [241]:
# # Pass the input data through the pre-trained model to get the embeddings
# all_embeddings = []
# for batch in dataset:
#     # with tf.device('/GPU:0'):
#     outputs = model(batch)
#     embeddings = outputs.pooler_output
#     all_embeddings.append(embeddings.numpy())

# # Concatenate the embeddings for all the batches
# all_embeddings = np.concatenate(all_embeddings, axis=0)

In [242]:
# all_embeddings.shape

In [243]:
# all_embeddings[0]

In [244]:
# # Pass the input data through the pre-trained model to get the embeddings
# for batch in dataset:
#     with tf.device('/GPU:0'):
#         outputs = model(batch)
#         embeddings = outputs.last_hidden_state
#         pooled_output = outputs.pooler_output

In [245]:
# outputs

In [246]:
# embeddings.shape

In [247]:
# tokens = tokenizer.tokenize(input_text)
# input_ids = tokenizer.convert_tokens_to_ids(tokens)
# # Create a `tf.data.Dataset` object to feed the input data to the model
# input_ids = tf.expand_dims(input_ids, axis=0)
# dataset = tf.data.Dataset.from_tensor_slices((input_ids,))
# dataset = dataset.batch(1)
# for batch in dataset:
#     with tf.device('/GPU:0'):
#         outputs = model(batch)
#         embeddings = outputs.last_hidden_state
#         pooled_output = outputs.pooler_output

In [248]:
# same_1 = mtokenize3(X_train[title].values[0])

In [249]:
# embedding1 = X_train_embeddings[0]
# embedding2 = all_embeddings[0]

# # Compute the cosine similarity between the embeddings
# cosine_similarities = tf.keras.losses.cosine_similarity(embedding1, embedding2)

# # Get the distance between the embeddings by subtracting the cosine similarity from 1
# distance = 1 - cosine_similarities

# print(distance)

In [250]:

# X_train_embeddings = [mtokenize3(word) for word in X_train[title]]
# X_test_embeddings = [mtokenize3(word) for word in X_test[title]]


In [260]:

X_train_embeddings = []
X_test_embeddings = []
X_whole_title = []
X_whole_bull_pt = []
X_whole_desc = []


In [261]:
# for i in data[title]:
#     X_whole_title.append(mtokenize3(i))

In [262]:
for i in range(len(data[title].values)):
    # X_whole_title.append(mtokenize3(data[title].values[i]))
    X_whole_title.append(mtokenize3(data[title].values[i]))
    X_whole_bull_pt.append(mtokenize3(data[bull_pt].values[i]))
    X_whole_desc.append(mtokenize3(data[desc].values[i]))

In [263]:
X_whole_title = np.array(X_whole_title)
X_whole_bull_pt = np.array(X_whole_bull_pt)
X_whole_desc = np.array(X_whole_desc)

In [266]:
X_whole_title.shape

(5000, 768)

In [265]:
# # add the X_whole to the data
# data['embeddings'] = X_whole
X_whole_title = np.reshape(X_whole_title, (5000, 768))
X_whole_bull_pt = np.reshape(X_whole_bull_pt, (5000, 768))
X_whole_desc = np.reshape(X_whole_desc, (5000, 768))


In [267]:

data_v2 = pd.concat([data.reset_index(), pd.DataFrame(X_whole_title).reset_index()], axis=1, ignore_index=True)
data_v2 = pd.concat([data.reset_index(), pd.DataFrame(X_whole_bull_pt).reset_index()], axis=1, ignore_index=True)
data_v2 = pd.concat([data.reset_index(), pd.DataFrame(X_whole_desc).reset_index()], axis=1, ignore_index=True)
data_v2

,0,1,2,3,4,5,6,7,8,9,...,2382,2383,2384,2385,2386,2387,2388,2389,2390,2391
0,1896156,Bildos 3Ply Non-Woven Fabric Disposable Surgic...,"[Care Instructions: Non Washable,Mask Has 3 La...","Face masks may also be called isolation, laser...",669.29,0,0,0,0,0,...,0.51,0.40,0.81,-0.05,-0.15,0.23,0.56,-0.87,-0.45,0.38
1,1715725,Negaor 10 PCS Interior Door Card Trim Panel Re...,[* Interior door card trim panel clips fastene...,Please check the Manufacturer Part Number and ...,314.96,0,0,0,0,0,...,0.91,-0.94,0.99,0.38,-0.84,-0.63,0.94,-0.96,-0.79,0.52
2,229576,"Madden Girl Women's Domain Boot, Black Fabric,...",Chukka-inspired boot with four-eye lacing syst...,Suede wedge laceup bootie,500.00,0,0,0,0,0,...,0.18,-0.52,0.97,0.79,-0.42,0.49,0.24,-0.74,-0.73,0.87
3,670308,Patel's Velocity perfume (60ml)(pack of 1),"[Long Lasting Fragrances,Made From Real and Na...",Patel's Velocity Perfume is Long Lasting Perfu...,157.48,0,0,0,0,0,...,0.74,-0.42,0.97,0.52,-0.33,-0.16,0.82,-0.95,-0.58,0.35
4,945067,WorldCare® 1000W Car Power ply Output Socket U...,"[New & Imported Item,Delivers within 3 to 5 we...",WorldCare 1000W Car Power ply Output Socket US...,590.55,0,0,0,0,0,...,0.54,-0.04,0.86,0.85,-0.07,0.12,0.64,-0.68,-0.66,0.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,380368,Gadget Gear Vinyl Skin Back Sticker Sticker Bo...,[3d Textured Mobile Skin: GadgetGear skins dis...,A GadgetGear skin comes in the true-textured f...,700.00,0,0,0,0,0,...,0.79,-0.83,0.99,0.20,-0.56,-0.48,0.81,-0.98,-0.49,-0.03
4996,688391,VICKYBEN Women's A-line Tulle Prom Formal Even...,"[Fabric: High quality soft tulle, comfortable ...",<b> Welcome to VICKYBEN Store </b> <br> More d...,590.55,0,0,0,0,0,...,0.79,-0.73,0.96,0.65,-0.12,-0.11,0.82,-0.93,-0.76,0.67
4997,2076124,Chhaya Hanging Hook Planter Railing Flower Pla...,"[Dimensions: Diameter 22 cm Height 28 cm,Set o...",Chhaya Nursery designer plastic planter pots a...,1102.36,0,0,0,0,0,...,0.52,-0.82,0.99,0.68,-0.49,-0.54,0.62,-0.98,-0.76,0.59
4998,908550,Luxocase Realme X2 Pro Carbon Fiber Leather Bl...,"[Premium matte finish, stylish, and durable sn...",LUXOCASE PRESENT PRINTED DESIGNER BACK CASE CO...,511.81,0,0,0,0,0,...,0.41,-0.42,0.98,0.82,-0.55,0.41,0.46,-0.90,-0.46,0.76


In [268]:
data_v2.shape

(5000, 2392)

In [269]:
data = data.drop(columns=[desc, bull_pt])

In [273]:
data_v3 = data_v2.drop(columns=[0,1,2,3])


In [274]:

# data_v3 = data_v2.drop(columns=[0])
data_v3.head()

,4,5,6,7,8,9,10,11,12,13,...,2382,2383,2384,2385,2386,2387,2388,2389,2390,2391
0,669.29,0,0,0,0,0,0,0,0,0,...,0.51,0.40,0.81,-0.05,-0.15,0.23,0.56,-0.87,-0.45,0.38
1,314.96,0,0,0,0,0,0,0,0,0,...,0.91,-0.94,0.99,0.38,-0.84,-0.63,0.94,-0.96,-0.79,0.52
2,500.00,0,0,0,0,0,0,0,0,0,...,0.18,-0.52,0.97,0.79,-0.42,0.49,0.24,-0.74,-0.73,0.87
3,157.48,0,0,0,0,0,0,0,0,0,...,0.74,-0.42,0.97,0.52,-0.33,-0.16,0.82,-0.95,-0.58,0.35
4,590.55,0,0,0,0,0,0,0,0,0,...,0.54,-0.04,0.86,0.85,-0.07,0.12,0.64,-0.68,-0.66,0.83


In [275]:
target_col_name = 4

In [ ]:
# for i in X_train[title]:
#     X_train_embeddings.append(mtokenize3(i))

In [ ]:
# X_train_embeddings = np.array(X_train_embeddings)

In [ ]:
# X_train_embeddings.shape

In [ ]:
# for i in X_test[title]:
#     X_test_embeddings.append(mtokenize3(i))

In [ ]:
# X_test_embeddings = np.array(X_test_embeddings)

In [ ]:
# X_test_embeddings.shape

In [276]:

# Split data
# X_train, X_test, y_train, y_test = train_test_split(data.drop([prod_len], axis=1), data[prod_len], test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(data_v3.drop([target_col_name], axis=1), data_v3[target_col_name], test_size=0.2, random_state=42)


In [ ]:
# print(X_train_embeddings[0])
# import pickle
# pickle.dump(X_train_embeddings, open("X_train_embeddings.pkl", "wb"))
# pickle.dump(X_test_embeddings, open("X_test_embeddings.pkl", "wb"))

In [ ]:
# import pickle


In [ ]:

# X_train_embeddings = pickle.load(open("X_train_embeddings.pkl", "rb"))
# X_test_embeddings = pickle.load(open("X_test_embeddings.pkl", "rb"))

In [ ]:
# %pip install nltk

In [ ]:

# X_train_embeddings2 = [w2v_model.wv[word] for word in X_train[desc]]
# X_test_embeddings2 = [w2v_model.wv[word] for word in X_test[desc]]


In [ ]:
# X_train_embeddings = np.reshape(X_train_embeddings, (800, 768))

In [ ]:
# X_train_embeddings.shape

In [ ]:
X_test_embeddings = np.reshape(X_test_embeddings, (200, 768))

In [ ]:
mdf2 = pd.DataFrame(X_train_embeddings)

In [ ]:
X_train.shape

In [ ]:
mdf2.shape

In [ ]:
mdf3_mergev2 = X_train.copy()

In [ ]:
mdf3_mergev2 = mdf3_mergev2.reset_index()

In [ ]:
mdf2 = mdf2.reset_index()

In [ ]:
# for i in mdf2.columns.values:
#     # mdf3_mergev2[i] = mdf2[i]
#     mdf3_mergev2 = mdf3_mergev2.join(mdf2[i])

mdf3_mergev2 = pd.concat([mdf3_mergev2, mdf2], axis=1)
mdf3_mergev2

In [ ]:
mdf3_mergev2.shape

In [ ]:
mdf3_mergev2 = mdf3_mergev2.drop(columns=[title, desc, bull_pt, 'index'])
mdf3_mergev2

In [ ]:
mdf3_mergev1 = pd.concat([X_train, mdf2], axis=1, ignore_index=True)

In [ ]:
mdf3_mergev1.shape

In [ ]:
# X_train_final = None
# X_test_final = None
# # Merge embeddings with other features
# X_train_final = pd.concat([X_train.drop([title, desc, bull_pt], axis=1), mdf2], axis=1, ignore_index=True)

X_train_final = mdf3_mergev2

In [ ]:

X_test_final = pd.concat([X_test.drop([title, desc, bull_pt], axis=1).reset_index(), pd.DataFrame(X_test_embeddings).reset_index()], axis=1, ignore_index=True)


In [ ]:
X_test_final

In [ ]:
X_train.head()

In [ ]:
X_train_final.shape

In [ ]:
X_train_final.shape

In [ ]:
y_train

In [277]:
X_train_final_v2 = X_train_final.copy()
X_test_final_v2 = X_test_final.copy()

# X_train_final_v2["missing_features"] = 0
# X_test_final_v2["missing_features"] = 0

# X_train_final_v2["missing_features2"] = 0
# X_test_final_v2["missing_features2"] = 0

NameError: name 'X_train_final' is not defined

In [ ]:
X_train_final_v3 = X_train_final_v2.copy()
X_test_final_v3 = X_test_final_v2.copy()

# X_train_final_v2["missing_features"] = 0
# X_test_final_v2["missing_features"] = 0

# X_train_final_v2["missing_features2"] = 0
# X_test_final_v2["missing_features2"] = 0

In [ ]:
cols = [range(800,1339)]
df.drop(df.columns[cols],axis=1,inplace=True)

In [ ]:
print(X_train_final_v2.shape)
print(X_test_final_v2.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
X_train_final_v2.head()
col_values = X_train_final_v2.columns.values
print(col_values)

In [ ]:
X_test_final_v2.head()
col_vtest = X_test_final_v2.columns.values
print(col_vtest)

In [ ]:
X_train_final_v2 = X_train_final_v2.rename(columns={
    col_values[i] : col_vtest[i] for i in range(len(col_values))
})

In [ ]:
X_test_final_v2.head()

In [ ]:
# X_test_final_v2.drop(columns=[1339, 1340], inplace=True)

In [278]:
X_train_final_v2 = X_train.copy()
X_test_final_v2 = X_test.copy()

# y_train = y_train.copy()
# y_test 

In [279]:
print(X_train_final_v2.shape)
print(X_test_final_v2.shape)
print(y_train.shape)
print(y_test.shape)

(4000, 2387)
(1000, 2387)
(4000,)
(1000,)


In [282]:
X_train_final_v2.head()

,0,1,2,3,4,5,6,7,8,9,...,2377,2378,2379,2380,2381,2382,2383,2384,2385,2386
4227,0,0,0,0,0,0,0,0,0,0,...,0.55,-0.38,0.92,0.17,-0.23,-0.17,0.57,-0.92,-0.58,0.19
4676,0,0,0,0,0,0,0,0,0,0,...,0.05,-0.59,0.92,0.66,-0.55,0.66,0.27,-0.82,-0.31,0.12
800,0,0,0,0,0,0,0,0,0,0,...,0.31,-0.22,0.94,0.80,-0.34,0.15,0.40,-0.93,-0.57,0.74
3671,0,0,0,0,0,0,0,0,0,0,...,0.60,-0.83,0.97,0.62,-0.89,-0.24,0.68,-0.83,-0.59,0.23
4193,0,0,0,0,0,0,0,0,0,0,...,0.45,-0.57,0.97,0.75,-0.37,0.20,0.50,-0.95,-0.71,0.74


In [ ]:
X_test_final_v2.head()

In [ ]:
X_train_final_v2 = X_train_final_v2.drop(columns = [1])
X_test_final_v2 = X_test_final_v2.drop(columns = [1])


In [281]:
X_test_final_v2 = X_test_final_v2.rename(columns={
    X_test_final_v2.columns.values[i] : i for i in range(len(X_test_final_v2.columns.values))
})
X_train_final_v2 = X_train_final_v2.rename(columns={
    X_train_final_v2.columns.values[i] : i for i in range(len(X_train_final_v2.columns.values))
})

In [283]:
from sklearn.metrics import mean_absolute_percentage_error

def score(y_test, preds):
    error = mean_absolute_percentage_error(y_test, preds)
    return max(0, 100*(1 - error))

In [284]:
from lazypredict.Supervised import LazyRegressor
# # Merge embeddings with other features
# X_train_final = pd.concat([X_train.drop([desc], axis=1), pd.DataFrame(X_train_embeddings2)], axis=1)
# X_test_final = pd.concat([X_test.drop([desc], axis=1), pd.DataFrame(X_test_embeddings2)], axis=1)

# Use Lazy Predict
# clf = LazyClassifier(verbose=2,ignore_warnings=False, custom_metric=None)
# clf = LazyRegressor(verbose=2,ignore_warnings=False, custom_metric=None)
# clf = LazyRegressor(verbose=2,ignore_warnings=False, custom_metric=None)
clf = LazyRegressor(verbose=2, ignore_warnings=False, custom_metric=score)
# clf = LazyRegressor(verbose=2,ignore_warnings=False, custom_metric=None,input_features=list(X_train_final_v2.columns) ) # throws error
models,predictions = clf.fit(X_train_final_v2, X_test_final_v2, y_train, y_test)

print(models)

# try scaling data first


  2%|▏         | 1/42 [00:22<15:30, 22.70s/it]

{'Model': 'AdaBoostRegressor', 'R-Squared': 0.22351081930192174, 'Adjusted R-Squared': 1.558870815214251, 'RMSE': 6114.73812362015, 'Time taken': 22.70244812965393, 'score': 0}


  2%|▏         | 1/42 [00:55<38:04, 55.71s/it]


KeyboardInterrupt: 

In [ ]:
# from lazypredict.Supervised import LazyRegressor
# from sklearn import datasets
# from sklearn.utils import shuffle
# import numpy as np

# boston = datasets.load_boston()
# X, y = shuffle(boston.data, boston.target, random_state=13)
# X = X.astype(np.float32)

# offset = int(X.shape[0] * 0.9)

# X_train, y_train = X[:offset], y[:offset]
# X_test, y_test = X[offset:], y[offset:]

# reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)
# models, predictions = reg.fit(X_train, X_test, y_train, y_test)

# print(models)

In [ ]:
X_train_final

In [ ]:
models

In [ ]:
predictions

In [ ]:

X_train_embeddings2 = [w2v_model.wv[word] for word in X_train[desc]]
X_test_embeddings2 = [w2v_model.wv[word] for word in X_test[desc]]

# Merge embeddings with other features
X_train_final = pd.concat([X_train.drop([title], axis=1), pd.DataFrame(X_train_embeddings)], axis=1)
X_test_final = pd.concat([X_test.drop([title], axis=1), pd.DataFrame(X_test_embeddings)], axis=1)

# Merge embeddings with other features
X_train_final = pd.concat([X_train.drop([desc], axis=1), pd.DataFrame(X_train_embeddings2)], axis=1)
X_test_final = pd.concat([X_test.drop([desc], axis=1), pd.DataFrame(X_test_embeddings2)], axis=1)

# Use Lazy Predict
clf = LazyClassifier(verbose=2,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train_final, X_test_final, y_train, y_test)

print(models)


In [ ]:
def mtokenize2(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors='tf', max_length=512)
    outputs = model(inputs)
  
    return outputs[0]

In addition to outputs.last_hidden_state, TFBaseModelOutputWithPoolingAndCrossAttentions also contains a pooler_output tensor, which is a fixed-size vector that summarizes the entire input sequence and is often used as input to a downstream task-specific classifier. The size of this pooler_output tensor is typically equal to the hidden size of the pre-trained model. Additionally, if the pre-trained model has cross-attention layers, TFBaseModelOutputWithPoolingAndCrossAttentions may also contain cross-attention tensors that represent the attention weights applied to the input sequence from other sequences in the batch.

In the case of the TFBaseModelOutputWithPoolingAndCrossAttentions class from the Hugging Face Transformers library, outputs.last_hidden_state refers to a tensor containing the contextualized embeddings of each token in the input sequence.


In [ ]:
def mtokenize(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors='tf', max_length=512)
    outputs = model(inputs)
    

    output_array = outputs[0].numpy()

    # Flatten and reshape the output array to match sklearn input format
    output_array = output_array.flatten().reshape(1, -1)
    
    # return outputs[0]
    return output_array

In [ ]:
texts = []
for i in range(3):
    mtest = X_train[title].values[i] + " " + X_train[desc].values[i] + " " + X_train[bull_pt].values[i]
    texts.append(mtest)

In [ ]:
for i in texts:
    # print(mtokenize(i))
    # print(mtokenize(i).shape)

    # print(mtokenize2(i))
    # print(mtokenize2(i).shape)

    print(mtokenize3(i))
    print(mtokenize3(i).shape)

In [ ]:
mtokenize("hansco this")

In [ ]:
mtokenize("hansco this")[0].shape

In [ ]:
%pip freeze > requirements.txt

In [ ]:
%pip install lazypredict


In [285]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import pandas as pd

# Load data into a pandas DataFrame
# data = pd.read_csv('data.csv')

# Split data into features (X) and target variable (y)
# X = data.drop('target_variable', axis=1)
# y = data['target_variable']

# Split data into training and test sets
# X_train1, X_test2, y_train3, y_test4 = train_test_split(X, y, test_size=0.2)

# Create a RandomForestRegressor object with default hyperparameters
rf_regressor = RandomForestRegressor()

# Fit the regressor to the training data
rf_regressor.fit(X_train_final_v2, y_train)

# Make predictions on the test data
predictions = rf_regressor.predict(X_test_final_v2)

# Evaluate the model using RMSE
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
print('RMSE:', rmse)

# shot in the dark is not going to work. maybe first visualize the data and see if there is a pattern
# let's see what you can understand from it

# that's off, very off
# map it maybe?


RMSE: 13756.648203570148
